In [3]:
import pandas as pd
import numpy as np
import os
import json
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# --- CONFIG ---
DATA_PATHS = {
    2021: "datasets/LLCP2021.XPT_",
    2022: "datasets/LLCP2022.XPT_",
    2023: "datasets/LLCP2023.XPT_",
    2024: "datasets/LLCP2024.XPT_"
}

MAPPING_JSON = "datasets/brfss_variable_map.json"

STANDARD_FEATURES = [
    "diabetes", "sex", "age_group", "education", "income",
    "employment", "marital", "exercise_any", "smoke_status", "sleep_hours",
    "bmi", "general_health", "physical_health_days",
    "mental_health_days", "poor_health_days",
    "hypertension", "cholesterol_check",
    "heart_attack", "coronary_hd"
]

# --- HELPER FUNCTIONS ---

def load_mapping(path=MAPPING_JSON):
    with open(path, "r") as f:
        return json.load(f)

def map_target(x):
    if x in [1, 4]:
        return 1
    elif x == 3:
        return 0
    else:
        return np.nan

def clean_column(name, series):
    s = series.copy()
    s.replace([7, 77, 777, 7777, 9, 99, 999, 9999], np.nan, inplace=True)

    if name == "exercise_any":
        return s.replace({1: 1, 2: 0})
    if name == "smoke_status":
        return s.replace({1:1, 2:1, 3:0})
    if name in ["hypertension", "cholesterol_check", "heart_attack", "coronary_hd"]:
        return s.replace({1:1, 2:0, 3:0})

    return s

def process_year(path, year, mapping):
    print(f"\nProcessing year {year} at {path}")
    df = pd.read_sas(path, format="xport")

    year_map = mapping[str(year)]
    use_cols = {}
    for std in STANDARD_FEATURES:
        if std in year_map:
            use_cols[std] = year_map[std]
        else:
            print(f" Warning: standard feature '{std}' not mapped for year {year}")

    actual_cols = [col for col in use_cols.values() if col in df.columns]
    df_sel = df[actual_cols].copy()
    rename_dict = {v: k for k, v in use_cols.items()}
    df_sel.rename(columns=rename_dict, inplace=True)

    df_sel["diabetes"] = df_sel["diabetes"].map(map_target)


    df_sel = df_sel[df_sel["diabetes"].notna()]

    # Clean each column
    for col in df_sel.columns:
        df_sel[col] = clean_column(col, df_sel[col])

    # Convert BMI if exists
    if "bmi" in df_sel.columns:
        df_sel["bmi"] = df_sel["bmi"] / 100.0

    df_sel["year"] = year
    print(f"Year {year} cleaned: {df_sel.shape[0]} rows, {df_sel.shape[1]} cols")
    return df_sel

def main():
    mapping = load_mapping()
    all_dfs = []
    for year, path in DATA_PATHS.items():
        if os.path.exists(path):
            df_year = process_year(path, year, mapping)
            all_dfs.append(df_year)
        else:
            print(f"File missing for year {year}: {path}")

    if not all_dfs:
        print("No datasets processed. Exiting.")
        return

    df_all = pd.concat(all_dfs, ignore_index=True)
    print(f"\nCombined shape: {df_all.shape}")


    out_path = "BRFSS_2021_2024_diabetes_ready_v2.csv"
    df_all.to_csv(out_path, index=False)
    print(f"Saved full cleaned dataset to: {out_path}")

if __name__ == "__main__":
    main()



Processing year 2021 at datasets/LLCP2021.XPT_
Year 2021 cleaned: 433900 rows, 18 cols

Processing year 2022 at datasets/LLCP2022.XPT_
Year 2022 cleaned: 440209 rows, 16 cols

Processing year 2023 at datasets/LLCP2023.XPT_
Year 2023 cleaned: 429086 rows, 16 cols

Processing year 2024 at datasets/LLCP2024.XPT_
Year 2024 cleaned: 453241 rows, 15 cols

Combined shape: (1756436, 19)
Saved full cleaned dataset to: BRFSS_2021_2024_diabetes_ready_v2.csv
